# Association between Fertility & Local Municipal Characteristics in Denmark 

### Imports

In [ ]:
import geopandas as gpd # for geospatial data handling
import pandas as pd
import contextily as cx # for plotting
import matplotlib.pyplot as plt # for plotting
from pyproj import CRS # for more advanced CRS modifications and transformations

import warnings
warnings.filterwarnings("ignore", category=DeprecationWarning) 
warnings.filterwarnings("ignore", category=FutureWarning) 

### Data

Load the data from DenmarkStatistik and geometries for municipalities of Denmark and then merge them into one geo dataframe

'Data/Educational_attainment_by_highest_education_completed_(total)_2021.xlsx'


In [ ]:
data_edu = pd.read_excel('Data/Educational_attainment_by_highest_education_completed_(total)_2021.xlsx')
data_fer = pd.read_excel('Data/fertility_rates_by_municipality_2021.xlsx')
data_bir = pd.read_excel('Data/live_births_by_municipality_2021.xlsx')
data_mar = pd.read_excel('Data/market_value_for_real_estate_average_dkk_2021.xlsx')

In [ ]:
geometries = gpd.read_file('Data/municipalities_dk.gpkg') #added
# geometry.head()

In [ ]:
# rename the columns to prepare for merge
data_edu.rename(columns = {'Unnamed: 0':'municipality_nm', 2021:'edu_att'}, inplace = True)
data_fer.rename(columns = {'Unnamed: 0':'municipality_nm', 2021:'fer_rate'}, inplace = True)
data_bir.rename(columns = {'Unnamed: 0':'municipality_nm', 2021:'live_birth'}, inplace = True)
data_mar.rename(columns = {'Unnamed: 0':'municipality_nm', 2021:'market_val'}, inplace = True)

In [ ]:
# merge to one data frame
data = pd.merge(pd.merge(data_fer, data_bir), pd.merge(data_edu, data_mar))
# data.head()

In [ ]:
data['municipality_nm'] = data['municipality_nm'].replace('Copenhagen', 'København')
geometries = geometries.rename(columns={'navn': 'municipality_nm'}) #added

In [ ]:
data = pd.merge(data, geometries[['municipality_nm', 'kommunekode', 'municipal_id']]) #added

In [ ]:
gdf = gpd.GeoDataFrame(data, geometry=geometries.geometry, crs = 'EPSG:25832') #added

,municipality_nm,fer_rate,live_birth,edu_att,market_val,kommunekode,region,municipal_id,geometry
0,København,1 563.9,10 285,501 212,2 399 736,0101,NaN,101,"MULTIPOLYGON (((723090.010 6171801.780, 723090..."
1,Frederiksberg,1 787.8,1 534,74 788,3 435 510,0147,NaN,147,"MULTIPOLYGON (((722290.770 6177984.800, 722299..."
2,Dragør,2 510.9,146,9 074,4 014 690,0155,NaN,155,"MULTIPOLYGON (((729575.650 6168529.720, 729555..."
3,Tårnby,2 056.7,473,28 727,2 823 345,0185,NaN,185,"MULTIPOLYGON (((729600.300 6172549.660, 729596..."
4,Albertslund,1 608.7,274,18 664,2 931 561,0165,NaN,165,"MULTIPOLYGON (((711377.280 6178342.550, 711352..."


## Inspecting spatial data and EDA

In [ ]:
# just ploting the plygons
cities.plot();

In [ ]:
# quick interactive map
cities.explore()

## Morans I